In [ ]:
import time
import torch
import torchvision
import cv2
import PIL.Image

import torchvision.transforms as transforms
import torch.nn.functional as F
import numpy as np

from jetbot import Robot
from torch2trt import TRTModule

device = torch.device('cuda')

In [ ]:
collision_model = TRTModule()
collision_model.load_state_dict(torch.load('best_collision_model_resnet18_trt.pth'))

steering_model = TRTModule()
steering_model.load_state_dict(torch.load('best_steering_model_xy_trt.pth'))

In [ ]:
mean = torch.Tensor([0.485, 0.456, 0.406]).cuda().half()
std = torch.Tensor([0.229, 0.224, 0.225]).cuda().half()

normalize = torchvision.transforms.Normalize(mean, std)

def preprocess(image):
    image = PIL.Image.fromarray(image)
    image = transforms.functional.to_tensor(image).to(device).half()
    image.sub_(mean[:, None, None]).div_(std[:, None, None])
    return image[None, ...]

In [ ]:
import traitlets
from IPython.display import display
import ipywidgets
from jetbot import Camera, bgr8_to_jpeg

# Load camera and link to Image widget
camera = Camera.instance()
image = ipywidgets.Image()
camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

# sliders for controlling steering
speed_gain_slider = ipywidgets.FloatSlider(min=0.0, max=0.5, step=0.01, description='speed gain')
steering_gain_slider = ipywidgets.FloatSlider(min=0.0, max=1.0, step=0.01, value=0.2, description='steering gain')
steering_dgain_slider = ipywidgets.FloatSlider(min=0.0, max=0.5, step=0.001, value=0.0, description='steering kd')
steering_bias_slider = ipywidgets.FloatSlider(min=-0.3, max=0.3, step=0.01, value=0.0, description='steering bias')

# Sliders to show model results:
# Probability of blockage
blocked_slider = ipywidgets.FloatSlider(description='blocked', min=0.0, max=1.0, orientation='vertical')
# Location of middle of the road (X,Y)
x_slider = ipywidgets.FloatSlider(min=-1.0, max=1.0, description='x')
y_slider = ipywidgets.FloatSlider(min=0, max=1.0, orientation='vertical', description='y')
# Steering strength
steering_slider = ipywidgets.FloatSlider(min=-1.0, max=1.0, description='steering')
# Current speed
speed_slider = ipywidgets.FloatSlider(min=0, max=1.0, orientation='vertical', description='speed')

print("Steering Control Widgets:")
display(speed_gain_slider, steering_gain_slider, steering_dgain_slider, steering_bias_slider)
print("")
print("Model results Visualization:")
display(ipywidgets.HBox([blocked_slider,image,y_slider, speed_slider]))
display(x_slider, steering_slider)

In [ ]:
# initialize robot
robot = Robot()

In [ ]:
angle = 0.0
angle_last = 0.0

def execute(change):
    global angle, angle_last
    image = change['new']
    image = preprocess(image)

    # collision avoidance
    blocked = collision_model(image)
    blocked = F.softmax(blocked, dim=1)

    prob_blocked = float(blocked.flatten()[0])

    blocked_slider.value = prob_blocked

    if prob_blocked > 0.80:
        robot.stop()
    else:
        # steering
        xy = steering_model(image).detach().float().cpu().numpy().flatten()
        x = xy[0]
        y = (0.5 - xy[1]) / 2.0

        x_slider.value = x
        y_slider.value = y

        speed_slider.value = speed_gain_slider.value

        angle = np.arctan2(x, y)
        pid = angle * steering_gain_slider.value + (angle - angle_last) * steering_dgain_slider.value
        angle_last = angle

        steering_slider.value = pid + steering_bias_slider.value

        robot.left_motor.value = max(min(speed_slider.value + steering_slider.value, 1.0), 0.0)
        robot.right_motor.value = max(min(speed_slider.value - steering_slider.value, 1.0), 0.0)

camera.observe(execute, names='value')

In [ ]:
# Unlink execute from camera when finished.
camera.unobserve(execute, names='value')

time.sleep(0.1)  # add a small sleep to make sure frames have finished processing

robot.stop()

In [ ]:
camera.stop()